In [189]:
from utils import load_data

from matplotlib import pyplot as plt
plt.style.use('ggplot')

from mpl_toolkits.mplot3d import Axes3D

import numpy as np

from keras.layers import Input, RNN, SimpleRNN, LSTM, concatenate, Dense, Dropout
from keras.models import Model
from keras.regularizers import l2
from keras.preprocessing.sequence import pad_sequences

In [65]:
dataset = 'cora'

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(dataset)
num_nodes = adj.shape[0]
num_features = features.shape[1]
num_classes = y_train.shape[1]

print("Total number of nodes: {}".format(adj.shape[0]))
print("Training nodes: {}".format(len(np.argwhere(y_train == True))))
print("Validation nodes: {}".format(len(np.argwhere(y_val == True))))
print("Test nodes: {}".format(len(np.argwhere(y_test == True))))
print("Num nodes: {}".format(num_nodes))
print("num_features: {}".format(num_features))

adj = np.array(adj.todense())
features = np.array(features.todense())

Total number of nodes: 2708
Training nodes: 140
Validation nodes: 500
Test nodes: 1000
Num nodes: 2708
num_features: 1433


In [149]:
neighbour_features = pad_sequences(neighbour_features)
print(neighbour_features.shape)

(2708, 168, 1433)


In [160]:
neighbour_features_train = neighbour_features[train_mask]
features_train = features[train_mask]
labels_train = y_train[train_mask]

neighbour_features_val = neighbour_features[val_mask]
features_val = features[val_mask]
labels_val= y_val[val_mask]

neighbour_features_test = neighbour_features[test_mask]
features_test = features[test_mask]
labels_test = y_test[test_mask]

print(neighbour_features_train.shape)

(140, 168, 1433)


In [193]:
max_length = neighbour_features.shape[1]
rnn_units = 16
hidden_units = [16, 32]
p = 0.5
l2_weight = 0.01

neighbour_features_input = Input(shape=(max_length, num_features), name='neighbour_features')
neighbour_features_input_dropped = Dropout(p)(neighbour_features_input)
encoder_outputs = SimpleRNN(units=rnn_units, dropout=p, kernel_regularizer=l2(l2_weight))(neighbour_features_input_dropped)

node_features_input = Input(shape=(num_features,), name='node_features')
x_1 = Dense(hidden_units[0], activation='relu', kernel_regularizer=l2(l2_weight))(node_features_input)

merged = concatenate([encoder_outputs, x_1])
merged_dropped = Dropout(p)(merged)

predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_weight))(merged_dropped)

model = Model(inputs=[neighbour_features_input,node_features_input], outputs = predictions)

model.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
neighbour_features (InputLayer) (None, 168, 1433)    0                                            
__________________________________________________________________________________________________
dropout_29 (Dropout)            (None, 168, 1433)    0           neighbour_features[0][0]         
__________________________________________________________________________________________________
node_features (InputLayer)      (None, 1433)         0                                            
__________________________________________________________________________________________________
simple_rnn_41 (SimpleRNN)       (None, 16)           23200       dropout_29[0][0]                 
__________________________________________________________________________________________________
dense_35 (

In [194]:
batch_size = 128
num_epoch = 1000

model.fit(x=[neighbour_features_train, features_train], y=labels_train,
          validation_data=([neighbour_features_val, features_val], labels_val),
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1
          )

Train on 140 samples, validate on 500 samples
Epoch 1/1000
140/140 [==============================] - 4s 28ms/step - loss: 2.8164 - acc: 0.1857 - val_loss: 2.6412 - val_acc: 0.1440
Epoch 2/1000
140/140 [==============================] - 2s 15ms/step - loss: 2.7025 - acc: 0.1500 - val_loss: 2.5953 - val_acc: 0.1500
Epoch 3/1000
140/140 [==============================] - 2s 15ms/step - loss: 2.6625 - acc: 0.2071 - val_loss: 2.5549 - val_acc: 0.1720
Epoch 4/1000
140/140 [==============================] - 2s 15ms/step - loss: 2.5025 - acc: 0.2500 - val_loss: 2.5234 - val_acc: 0.1800
Epoch 5/1000
140/140 [==============================] - 2s 15ms/step - loss: 2.5408 - acc: 0.2500 - val_loss: 2.4933 - val_acc: 0.1880
Epoch 6/1000
140/140 [==============================] - 2s 15ms/step - loss: 2.4232 - acc: 0.2500 - val_loss: 2.4615 - val_acc: 0.1980
Epoch 7/1000
140/140 [==============================] - 2s 15ms/step - loss: 2.3798 - acc: 0.2714 - val_loss: 2.4296 - val_acc: 0.1980
Epoch 8/1

Epoch 61/1000
140/140 [==============================] - 2s 15ms/step - loss: 1.2138 - acc: 0.9000 - val_loss: 1.9377 - val_acc: 0.5660
Epoch 62/1000
140/140 [==============================] - 2s 17ms/step - loss: 1.2036 - acc: 0.9000 - val_loss: 1.9351 - val_acc: 0.5700
Epoch 63/1000
140/140 [==============================] - 2s 17ms/step - loss: 1.2273 - acc: 0.8643 - val_loss: 1.9324 - val_acc: 0.5780
Epoch 64/1000
140/140 [==============================] - 2s 17ms/step - loss: 1.1755 - acc: 0.9000 - val_loss: 1.9295 - val_acc: 0.5900
Epoch 65/1000
140/140 [==============================] - 2s 18ms/step - loss: 1.1666 - acc: 0.9143 - val_loss: 1.9254 - val_acc: 0.5960
Epoch 66/1000
140/140 [==============================] - 2s 17ms/step - loss: 1.2021 - acc: 0.8071 - val_loss: 1.9206 - val_acc: 0.5960
Epoch 67/1000
140/140 [==============================] - 2s 17ms/step - loss: 1.1400 - acc: 0.9071 - val_loss: 1.9158 - val_acc: 0.6040
Epoch 68/1000
140/140 [=========================

140/140 [==============================] - 2s 16ms/step - loss: 0.8550 - acc: 0.9786 - val_loss: 1.7925 - val_acc: 0.6580
Epoch 122/1000
140/140 [==============================] - 2s 15ms/step - loss: 0.9036 - acc: 0.9571 - val_loss: 1.7929 - val_acc: 0.6620
Epoch 123/1000
140/140 [==============================] - 2s 15ms/step - loss: 0.8744 - acc: 0.9786 - val_loss: 1.7933 - val_acc: 0.6640
Epoch 124/1000
140/140 [==============================] - 2s 15ms/step - loss: 0.8921 - acc: 0.9857 - val_loss: 1.7935 - val_acc: 0.6640
Epoch 125/1000
140/140 [==============================] - 2s 16ms/step - loss: 0.8969 - acc: 0.9500 - val_loss: 1.7925 - val_acc: 0.6620
Epoch 126/1000
140/140 [==============================] - 2s 16ms/step - loss: 0.9115 - acc: 0.9714 - val_loss: 1.7908 - val_acc: 0.6620
Epoch 127/1000
128/140 [==========================>...] - ETA: 0s - loss: 0.9333 - acc: 0.9609

KeyboardInterrupt: 